# Metaclasses
I know, this is not a design pattern.<br>
But I think metaclass is usable only with due regard to how it affects<br>
other components within the systems just like design patterns.<br>

## Normal class

In [5]:
from typing import Any


class Foo(type):
    
    def __call__(cls, *args: Any, **kwds: Any) -> Any:
        
        print("Is this really called?")
        cls = super().__call__(*args,**kwds)
        return cls

class Bar(metaclass=Foo):
    pass

f = Bar()


Is this really called?


As you can see, type's call dunder method invokes new() and init() dunder methods.<br>

### What is metaclass for?
It is essentially to customize instantiation of a class(not an object)

In [10]:
class Meta(type):
    def __new__(cls,name,bases,namespace):
        x = super().__new__(cls,name,bases,namespace)
        x.db = {}
        return x
    
class Foo(metaclass=Meta):
    pass

a = Foo()

b= Foo()
print(a.db)
print(b.db)
print(id(a.db) == id(b.db))
        

{}
{}
True


### Example #1  - Preventing Multiple Inheritance

In [12]:
class NoMultipleInheritanceBase(type):
    def __new__(cls,clsname,bases,clsdict):
        if len(bases)> 1:
            raise TypeError("Multiple Inheritance Detected!")
        return super().__new__(cls,clsname,bases,clsdict)
    

class Base(metaclass=NoMultipleInheritanceBase):
    pass

class A(Base):
    pass

class B(Base):
    pass

try:
    class C(A,B):
        pass
except TypeError:
    print("Not allowed action caught!")


Not allowed action caught!


### Example #2 - Decorators vs Metaclasses

Consider a very simple problem of code repetition.<br>
We want to debug class methods, what we want is whenever<br>
the method in the class is executed, it should print its fully qualified name.<br>

In [16]:
#Decorator version

from functools import wraps
from typing import Callable

def debug(func:Callable):
    @wraps(func)
    def wrapper(*args,**kwargs):
        print("Fully qualified name of method:",func.__qualname__)
        return func(*args,**kwargs)
    return wrapper

#class decorator
def debugmethods(cls):
    for key,val in vars(cls).items():
        if callable(val):
            #decorate 
            setattr(cls,key,debug(val))
    return cls

@debugmethods
class Foo:
    def bar(self):
        print("Bar!")

    def foo_bar(self):
        print("Foobar!")
    

foo = Foo()
foo.bar()
foo.foo_bar()

Fully qualified name of method: Foo.bar
Bar!
Fully qualified name of method: Foo.foo_bar
Foobar!


#### That works but...
- What if we want to apply this method to all subclasses which inherit this Foo class?

The problem is if we have many such subclasses, then in that case we don't like adding<br><br>

a decorator to each one separately.<br><br>

Let's Have a look at metaclass version


In [17]:
from functools import wraps

def debug(func):
    @wraps(func)
    def wrapper(*args,**kwargs):
        print("Full name of this method:",func.__qualname__)
        return func(*args,**kwargs)
    return wrapper

def debugmethods(cls):
    for key,val in vars(cls).items():
        if callable(val):
            setattr(cls,key,debug(val))
    return cls

class DebugMeta(type):
    def __new__(cls,clsname,bases,clsdict):
        cls = super().__new__(cls,clsname,bases,clsdict)
        cls = debugmethods(cls)
        return cls

class Base(metaclass = DebugMeta):pass

class Foo(Base):
    def bar(self):
        print("bar!")

class Bar(Foo):
    def foo(self):
        print("foo!")

b= Bar()
b.foo()

Full name of this method: Bar.foo
foo!
